# Text Representation
## BoW and n-grams

Empezaremos por la representación más clave en NLP, que es la Bolsa de palabras.

La bolsa de palabras es extremadamente fácil de crear y extremadamente usada como input vector en muchísimos algoritmos de lenguaje. Nos permitirá recuperar numpy y ver la primera interacción con sklearn.

## ¿En qué consiste?

Consiste en representar un input textual como un vector. ¿Qué valor tendrá cada casilla de un vector? Fácil, primero generaremos un vocabulario de nuestro train data. Luego, asignaremos un entero a cada palabra y ese entero será el índice de este vector. Finalmente, por cada frase, asignaremos en cada posición del vector el número de veces que aparece en frases.

![](https://i.imgur.com/EPG2zSK.jpg)

Vamos a ello paso a paso.

In [0]:
documents = [
    'frase en castellano',
    'english sentence',
    'esta frase no esta en english',
    'estos documentos tienen muy poco sentido',
    'el documento es un conjunto de frases',
    'Vim es mucho mejor que emacs.',
    'tabs vs spaces'
]

### ¿Usaremos todas las palabras?

No siempre. Ver stop-words.

¿Alguna cosilla más en el vocabulary? UNK, PAD tokens.

In [0]:
def generate_vocabulary_maps(docs):
    vocabulary = {}
    inverse_vocabulary = {}
    for doc in documents:
        for token in doc.split(' '):
            if token not in vocabulary:
                vocabulary[token] = len(vocabulary)
                inverse_vocabulary[len(inverse_vocabulary)] = token
    return vocabulary, inverse_vocabulary

In [0]:
vocabulary, inverse_vocabulary = generate_vocabulary_maps(documents)
print(vocabulary)

{'muy': 10, 'tienen': 9, 'el': 13, 'mucho': 21, 'frases': 19, 'que': 23, 'en': 1, 'emacs.': 24, 'english': 3, 'vs': 26, 'esta': 5, 'frase': 0, 'no': 6, 'documentos': 8, 'de': 18, 'conjunto': 17, 'sentence': 4, 'castellano': 2, 'estos': 7, 'mejor': 22, 'documento': 14, 'spaces': 27, 'Vim': 20, 'es': 15, 'tabs': 25, 'un': 16, 'sentido': 12, 'poco': 11}


## Introducción de numpy en el curso.


In [0]:
import numpy as np

In [0]:
new_representation_np = np.zeros((len(vocabulary)), dtype='int32')
new_representation_np

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=int32)

In [0]:
string = 'documento en castellano'

In [0]:
def transform(x, vocab):
    assert type(x) == str, 'wrong type. x must be a sentence'
    new_representation_np = np.zeros((len(vocab)), dtype='int32')
    idx = [vocab[token] for token in x.split(' ')]
    new_representation_np[idx] += 1
    return new_representation_np    

In [0]:
transform(string, vocab=vocabulary)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=int32)

In [0]:
new_docs = [
    'este documento es de programacion en castellano',
    'castellano documento en en',
]

In [0]:
def itransform(X, vocab):
    assert type(X)==list, 'X must be a list'
    rows= len(X)
    rep = np.zeros((rows, len(vocab)), dtype='int32')
    for i, x in enumerate(X):
        tokens = [vocab[token] for token in x.split(' ') if token in vocab]
        for t in tokens:
            rep[i, t] += 1
    return rep

In [0]:
itransform(new_docs, vocabulary)

array([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]], dtype=int32)

In [0]:
docs = documents + new_docs
count_docs = itransform(docs, vocabulary)

Ahora que ya hemos visto lo rápido que es preparar un modelo de Bolsa de palabras, habría que hacer un pequeno análisis de los problemas que puede tener.

Ver Bow Issues en OneNote.

¿Cuán grande puede ser un vocabulario?

El español tiene 100.000 palabras aproximadamente, sin contar con conjugaciones verbales.

<div align="center">
![](https://i.imgur.com/a29W4JC.png)
</div>

# N-grams

Una de las posibles soluciones al modelo de Bolsa de Palabras es una ampliación bastante natural. En lugar de coger palabras sueltas, cogemos n-palabras consecutivas. A esto lo denominamos n-grams.

Los bigrams, y trigrams son modelos de features bastante usados, hasta 5-grams, y no son de uso exclusivo. Es decir, podemos concatenar el input de una frase usando [unigram_vector, bigram_vector, trigram_vector].

p.e.:

> ¿Qué puedo hacer?

> Me dijo que quería hacer algo.

En un problema de clasificación, quizás es importante saber órdenes, aunque sean parciales.

La siguiente quote proviene de Google, cuando hicieron una release de los n-grams que calcularon en la web.

> 
We processed 1,024,908,267,229 words of running text and are publishing the counts for all 1,176,470,663 five-word sequences that appear at least 40 times. There are 13,588,391 unique words, after discarding words that appear less than 200 times.



Ahora veremos como se implementa un simple modelo de n-grams.

In [0]:
"""
<SOS>
<EOS>
"""
from collections import Counter

def compute_ngrams(docs, min_n=1, max_n=2, ngrams={}):
    assert max_n>min_n, 'max ngram must be bigger than min ngram'
    ngrams = ngrams if ngrams else {i:Counter() for i in range(min_n, max_n+1)} 
    for doc in docs:
        doc = '<SOS> ' + doc + ' <EOS>'
        tokenized_doc = doc.split(' ')
        for ix in range(len(tokenized_doc)):
            ngrams_doc = [" ".join(tokenized_doc[ix:ix+i]) for i in range(min_n, max_n+1) if ix+i < len(tokenized_doc)+1]
            for i, ngram in enumerate(ngrams_doc):
                ngrams[i+1][ngram]+=1
    return ngrams   

def update_ngrams(docs, ngrams, min_n=False, max_n=False):
    min_n = min_n if min_n else min(ngrams.keys())
    max_n = max_n if max_n else max(ngrams.keys())
    return compute_ngrams(docs, min_n, max_n, ngrams)
    
    

In [0]:
ngrams = compute_ngrams(docs)
for k in ngrams.keys():
    print(ngrams[k].most_common(2))
ngrams = update_ngrams(docs, ngrams)
for k in ngrams.keys():
    print(ngrams[k].most_common(2))

[('<EOS>', 9), ('<SOS>', 9)]
[('documento es', 2), ('castellano <EOS>', 2)]
[('<EOS>', 18), ('<SOS>', 18)]
[('documento es', 4), ('castellano <EOS>', 4)]


En este punto ya tenemos unas features básicas para poder realizar tareas de clasificación de textos con un clasificador como bayes.

Solo nos quedaría mapear esto a un numpy array. En estos ejemplos todo es magnífico. Pero, ¿y en la realidad? ¡Vectores enormes! ¡Explosión de features! ¿Qué pasa con los vocabularios muy extensos?

In [0]:
def generate_feature_maps(features):
    """
    Adapatar a las estructuras de cada uno, i.e cada proyecto.
    Podriamos estar interesados en el inverso.   
    """
    feature_map = {}
    for ngram in features.values():
        for token in ngram.keys():
            feature_map[token] = len(feature_map)
    return feature_map

In [0]:
feature_map = generate_feature_maps(ngrams)

In [0]:
print('Hay', len(docs), 'frases')
print('Hemos generado', len(feature_map), 'features')
print('Solo teniamos', len(set([token for doc in docs for token in doc.split(' ')])), 'palabras unicas')
print('Comprobacion con ngrams', len(ngrams[1])-2, 'quitando <SOS> y <EOS>')

Hay 9 frases
Hemos generado 82 features
Solo teniamos 30 palabras unicas
Comprobacion con ngrams 30 quitando <SOS> y <EOS>


In [0]:
def generate_feature_vector(docs, vocab, min_n=1, max_n=3):
    feature_vector = np.zeros((len(docs), len(vocab)), dtype='int32')
    for i, doc in enumerate(docs):
        doc = '<SOS> ' + doc + ' <EOS>' # tendria que estar hecho en el preproceso de la frase esto!
        tokenized_doc = doc.split(' ')
        print(tokenized_doc)
        for ix in range(len(tokenized_doc)):
            ngrams_doc = [" ".join(tokenized_doc[ix:ix+i]) for i in range(min_n, max_n+1) if ix+i < len(tokenized_doc)+1]
            print(ngrams_doc)
            maped = [vocab[ngram] for ngram in ngrams_doc if ngram in vocab]
            for ngram in maped:
                feature_vector[i, ngram] += 1
    return feature_vector

In [0]:
test_docs = ['nuevo documento']
test_vector = generate_feature_vector(test_docs, feature_map)

['<SOS>', 'nuevo', 'documento', '<EOS>']
['<SOS>', '<SOS> nuevo', '<SOS> nuevo documento']
['nuevo', 'nuevo documento', 'nuevo documento <EOS>']
['documento', 'documento <EOS>']
['<EOS>']


Se generan vectores enormes con muchos 0 y pocos 1. Más adelante veremos cómo dejar de tener vectores tan grandes. Hay dos tipos de representación en vectores: sparse, que es la que tenemos aquí, y dense, que la veremos en PCA/SVD o word embeddings.  

In [0]:
from scipy.sparse import csr_matrix
sparsified = csr_matrix(test_vector)
print(sparsified)

  (0, 6)	1
  (0, 24)	1
  (0, 31)	1


 <div><font color="red">

NLP tiene un gran problema que acabamos de ver. El vocabulario es ~infinito.</font>

El vocabulario se puede generar vocabulario usando tu train data, tambien se puede generar vocabulario con las palabras más usadas en un idioma. Es complicadísimo generar un vocabulario general para una tarea que contenga NLP.
</div>

De hecho, a lo largo de las sesiones, veremos que pasa con todas aquellas palabras Out Of Vocabulary (OOV). Es importante que tengamos en cuenta que esto nos va a ocurrir, y como modelamos nuestros problemas para que su afectación sea mínima, o nos permita generalizar de forma óptima.


 *Es MUY importante en nuestros pipelines de NLP intentar controlar el vocabulario de alguna manera.*

Hay varias maneras de hacerlo e iremos viendolas a lo largo de las sesiones. Ya hemos visto una (distancia de edición) para intentar corregir errores ortográficos. En problemas de clasificación con metodos de representación más clásicos, usaremos smoothing. En deep learning hay la tendencia de "unkificarlo todo", es decir todos aquellos tokens que desconocemos, tratarlos con un token especial < UNK >

A continuación veremos lo mismo de antes con la libreria sklearn, muy recomendable tirar de ella tanto para feature extraction como para clasificación.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [0]:
vectorizer = CountVectorizer(ngram_range=(1,3))
vectorizer.fit(documents)
vectorizer.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [0]:
print(vectorizer.transform(['nuevo documento']).toarray())
print(vectorizer.transform(['nuevo documento']))
vectorizer.transform(['nuevo documento'])

[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
  (0, 6)	1


<1x73 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

Es muy fácil de usar, bastante eficiente y 100% integrable con otras librerías, ya que devuelve numpy arrays, que son la base para muchas otras librerías.

En caso de que el vocabulario sea muy grande, en lugar de utilizar CountVectorizer usaríamos http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer .

Es más eficiente a nivel de memoria, pero con el drawback de que no podríamos sacar el inverso de las features que calcula. De todas formas, pronto veremos que los one hot vectors se usan cada vez menos. 

Normalmente, todos estas funciones como el CountVectorizer, o cualquier algoritmo que queramos implementar, debería permitir-nos poder usar tanto palabras, como carácteres como features. Aquí ya podemos verlo, y lo veremos mucho mejor en la siguiente sesión!